In [1]:
import sys
sys.path.append("/workspaces/chisel/")

# 👗 Example: Processing Fashion Brand NER (JSON Format) with Chisel

This example shows how to preprocess the explosion/ner-fashion-brands dataset into ChiselRecord objects for training transformer-based NER models using BILO labeling.

## 📥 Step 1: Load the Dataset

In [2]:
from datasets import load_dataset
ds = load_dataset("explosion/ner-fashion-brands")

/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 🧩 Step 2: Implement a JSON Span Parser
The dataset provides character-level spans in a spans field. We write a parser that extracts these into Chisel's EntitySpan format.

In [3]:
from typing import Tuple, List
from chisel.extraction.base.protocols import Parser
from chisel.extraction.models.models import EntitySpan

class JSONSpanParser(Parser):
    def parse(self, doc: dict) -> Tuple[str, List[EntitySpan]]:
        text = doc["text"]
        entities = [
            EntitySpan(
                text=text[e["start"]:e["end"]],
                start=e["start"],
                end=e["end"],
                label=e["label"]
            )
            for e in doc.get("spans", [])
        ]
        return text, entities

## 🔧 Step 3: Initialize Chisel Components

In [4]:
from chisel.extraction.tokenizers.hf_tokenizer import HFTokenizer
from chisel.extraction.span_aligners.token_span_aligner import TokenSpanAligner
from chisel.extraction.labelers.bilo_labeler import BILOLabeler
from chisel.extraction.labelers.label_encoder import SimpleLabelEncoder
from chisel.extraction.validators.validators import DefaultParseValidator, HFTokenAlignmentValidator
from chisel.extraction.models.models import ChiselRecord

## Component Setup

In [5]:
parser = JSONSpanParser()
tokenizer = HFTokenizer(model_name="bert-base-cased")
aligner = TokenSpanAligner()
labeler = BILOLabeler()

label_encoder = SimpleLabelEncoder(label_to_id={
    'O': 0,
    'B-FASHION_BRAND': 1,
    'I-FASHION_BRAND': 2,
    'L-FASHION_BRAND': 3,
    'U-FASHION_BRAND': 4,
})

parse_validators = [DefaultParseValidator()]
label_validators = [HFTokenAlignmentValidator(tokenizer=tokenizer.tokenizer)]

## 🔄 Step 4: Run the Preprocessing Pipeline

In [6]:
processed_data = []

for idx, example in enumerate(ds["train"]):
    text, entities = parser.parse(example)

    for validator in parse_validators:
        validator.validate(text, entities)

    tokens = tokenizer.tokenize(text)
    token_entity_spans = aligner.align(entities, tokens)

    labels = labeler.label(tokens, token_entity_spans)
    encoded_labels = label_encoder.encode(labels)

    for validator in label_validators:
        validator.validate(tokens, token_entity_spans)

    record = ChiselRecord(
        id=str(idx),
        chunk_id=0,
        text=tokenizer.tokenizer.decode([token.id for token in tokens]),
        tokens=tokens,
        input_ids=[token.id for token in tokens],
        attention_mask=[1] * len(tokens),
        entities=[tes.entity for tes in token_entity_spans],
        bio_labels=labels,
        labels=encoded_labels
    )
    processed_data.append(record)

Token indices sequence length is longer than the specified maximum sequence length for this model (525 > 512). Running this sequence through the model will result in indexing errors


### ✅ Output
You now have a list of ChiselRecord objects in processed_data, ready for training or export!

In [10]:
import json

for item in processed_data[0:5]:
    print(json.dumps(
        {
            "id": item.id,
            "text": item.text[0:100] + "...",
            "input_ids": item.input_ids[0:20],
            "attention_mask": item.attention_mask[0:20],
            "bio_labels": item.bio_labels[0:20],
            "labels": item.labels[0:20]
        },     indent=4, separators=(',', ': '))
    )

{
    "id": "0",
    "text": "It ' s all preference for which looks better, personally I feel that the more natural the hair looks...",
    "input_ids": [
        1135,
        112,
        188,
        1155,
        12629,
        1111,
        1134,
        2736,
        1618,
        117,
        7572,
        146,
        1631,
        1115,
        1103,
        1167,
        2379,
        1103,
        1716,
        2736
    ],
    "attention_mask": [
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1
    ],
    "bio_labels": [
        "O",
        "O",
        "O",
        "O",
        "O",
        "O",
        "O",
        "O",
        "O",
        "O",
        "O",
        "O",
        "O",
        "O",
        "O",
        "O",
        "O",
        "O",
        "O",
        "O"
    ],
    "labels": [
        0,